# Average premiumn predictor

Notebook‑ready script that trains a model to predict the **average** home‑insurance
premium per customer from a table of Canadian carrier quotes.

▸ Feature logic matches the hybrid scheme we discussed:
    • High‑card: postal FSA & city   → target‑encoded (CV‑safe)
    • Low‑card / binary flags       → one‑hot
    • Numeric & engineered fields   → passthrough
    • Apartment indicator           → custom FunctionTransformer

Change only the CSV path and (optionally) the customer‑ID recipe if
your dataset already contains a unique identifier.

## 0 ▸ Imports & setup

In [1]:
import numpy as np
import pandas as pd
import random
from pathlib import Path
from datetime import datetime

# machine learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.svm import SVR
# from xgboost import XGBRegressor # may need to brew install libomp for this to import
# from lightgbm import LGBMRegressor

# target encoding
from category_encoders.target_encoder import TargetEncoder

# persist model
import joblib

## 1 ▸ Load raw quote table

In [2]:
# Get list of all CSV files in the output directory
from pathlib import Path
output_dir = Path('../output')
csv_files = list(output_dir.glob('sample_data_*.csv'))

# Read and combine all CSV files
dfs = []
for i, file in enumerate(csv_files):
    temp_df = pd.read_csv(file)
    # Only keep header from first file
    if i > 0:
        temp_df = temp_df.iloc[1:]
    dfs.append(temp_df)

# Concatenate all dataframes
df = pd.concat(dfs, ignore_index=True)

# Trim column names
df.columns = df.columns.str.strip()

print(f"Combined {len(csv_files)} CSV files with a total of {len(df)} rows")

Combined 7 CSV files with a total of 356 rows


In [3]:
df.describe()

,unit_apt,move_in_year,occupants,num_claims,num_cancellations,num_fire_extinguishers,num_mortgages,phone,monthly_premium,annual_premium,...,527.0,2019,1,2,1.1,2.1,0,2897308160,148.33,1780.0
count,174.000000,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000,3.460000e+02,345.000000,345.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000e+01,10.00000,10.000000
mean,525.000000,2017.101156,1.959538,1.046243,0.976879,1.384393,1.000000,7.067180e+09,353.412638,4240.950725,...,520.200000,2015.500000,2.600000,1.500000,0.500000,2.300000,1.100000,5.357853e+09,649.51600,7794.200000
std,277.194095,4.621142,0.829586,0.811619,0.809034,1.108434,0.864769,2.292242e+09,235.400111,2824.801088,...,82.236515,1.900292,0.843274,0.527046,0.527046,0.483046,0.737865,2.593642e+09,258.37431,3100.537867
min,4.000000,2009.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.897308e+09,76.040000,912.000000,...,406.000000,2014.000000,1.000000,1.000000,0.000000,2.000000,0.000000,2.897308e+09,387.87000,4654.000000
25%,413.000000,2013.000000,1.000000,0.000000,0.000000,0.000000,0.000000,7.818398e+09,205.000000,2460.000000,...,436.250000,2014.250000,3.000000,1.000000,0.000000,2.000000,1.000000,2.897308e+09,455.95750,5471.500000
50%,625.500000,2017.000000,2.000000,1.000000,1.000000,2.000000,1.000000,8.554654e+09,277.500000,3330.000000,...,556.500000,2015.000000,3.000000,1.500000,0.500000,2.000000,1.000000,5.357853e+09,678.20500,8138.500000
75%,729.000000,2021.000000,3.000000,2.000000,2.000000,2.000000,2.000000,8.554654e+09,434.000000,5208.000000,...,586.000000,2015.000000,3.000000,2.000000,1.000000,2.750000,1.750000,7.818398e+09,686.14500,8233.750000
max,999.000000,2024.000000,3.000000,2.000000,2.000000,3.000000,2.000000,8.554654e+09,1896.670000,22760.000000,...,586.000000,2019.000000,3.000000,2.000000,1.000000,3.000000,2.000000,7.818398e+09,1294.42000,15533.000000


In [14]:
df[df["annual_premium"].isna()]

,id,timestamp,insurance_type,postal_code,address,unit_apt,city,province,name,date_of_birth,...,True.2,False.4,2.1,0,bigballs@gmail.com,2897308160,square_one,Square One Insurance Services,148.33,1780.0
345,11979aca-bbad-4a41-9c91-867fa67470ac,20250613_172455,H,M9R 1K8,30 Shadwick Dr,274.0,Etobicoke,ON,Derrick Lopez,1957-08-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,2.0,0.0,bigballs@gmail.com,2.897308e+09,sgi_canada,SGI Canada,435.00,5220.0
347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,2.0,0.0,bigballs@gmail.com,2.897308e+09,travelers_insurance,Travelers Insurance,684.08,8209.0
348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,2.0,1.0,bigballs@gmail.com,7.818398e+09,square_one,Square One Insurance Services,387.87,4654.0
349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,2.0,1.0,bigballs@gmail.com,7.818398e+09,max_insurance,Max Insurance,672.33,8068.0
350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,2.0,1.0,bigballs@gmail.com,7.818398e+09,sgi_canada,SGI Canada,685.83,8230.0
351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,2.0,1.0,bigballs@gmail.com,7.818398e+09,economical_mutual,Economical Mutual,686.25,8235.0
352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,2.0,1.0,bigballs@gmail.com,7.818398e+09,pembridge,Pembridge,1294.42,15533.0
353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,3.0,2.0,bigballs@gmail.com,2.897308e+09,max_insurance,Max Insurance,436.50,5238.0
354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,3.0,2.0,bigballs@gmail.com,2.897308e+09,sgi_canada,SGI Canada,514.33,6172.0


In [4]:
df.head()

,id,timestamp,insurance_type,postal_code,address,unit_apt,city,province,name,date_of_birth,...,True.2,False.4,2.1,0,bigballs@gmail.com,2897308160,square_one,Square One Insurance Services,148.33,1780.0
0,9b25309d-7bc5-42d1-88e0-848c3a1a7f5c,20250612_212036,H,M9A 3V3,1257 Royal York Rd,550.0,Etobicoke,ON,Alejandra Brown,1966-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9b25309d-7bc5-42d1-88e0-848c3a1a7f5c,20250612_212036,H,M9A 3V3,1257 Royal York Rd,550.0,Etobicoke,ON,Alejandra Brown,1966-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9b25309d-7bc5-42d1-88e0-848c3a1a7f5c,20250612_212036,H,M9A 3V3,1257 Royal York Rd,550.0,Etobicoke,ON,Alejandra Brown,1966-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9b25309d-7bc5-42d1-88e0-848c3a1a7f5c,20250612_212036,H,M9A 3V3,1257 Royal York Rd,550.0,Etobicoke,ON,Alejandra Brown,1966-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e71ffaa2-d1ae-43f3-8878-690d4fc01257,20250612_213054,H,M9P 3N3,24 Hartsdale Dr,NaN,Etobicoke,ON,Ashley Barron,1979-04-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2 ▸ Aggregate to one row per customer (average premium target)

In [5]:
feature_keep = [
    # raw categorical / numeric fields to engineer / encode
    "postal_code", "city", "province", "insurance_type", "unit_apt",
    "date_of_birth", "move_in_year",
    "num_fire_extinguishers", "num_mortgages", "occupants",
    "num_claims", "num_cancellations",
    # binary flags
    "has_sprinkler_system", "occupants_non_smokers",
    "active_home_insurance", "ever_insured", "multiline_discount",
    "has_monitored_fire_alarm", "has_deadbolt_locks",
    "has_monitored_burglar_alarm",
]

df_avg = (df
          .groupby("id", as_index=False)
          .agg({**{c: "first" for c in feature_keep},
                "annual_premium": "mean"})
          .rename(columns={"annual_premium": "avg_premium"}))

In [6]:
df_avg.describe()

,unit_apt,move_in_year,num_fire_extinguishers,num_mortgages,occupants,num_claims,num_cancellations,avg_premium
count,39.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000
mean,538.025641,2017.266667,1.373333,1.013333,2.053333,1.040000,1.026667,4321.416988
std,271.305565,4.768515,1.087687,0.862011,0.803596,0.829197,0.821556,2174.582327
min,4.000000,2009.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1938.500000
25%,418.000000,2013.000000,0.000000,0.000000,1.000000,0.000000,0.000000,3117.025000
50%,622.000000,2018.000000,2.000000,1.000000,2.000000,1.000000,1.000000,3705.466667
75%,725.500000,2021.000000,2.000000,2.000000,3.000000,2.000000,2.000000,4920.500000
max,999.000000,2024.000000,3.000000,2.000000,3.000000,2.000000,2.000000,14590.000000


## 3 ▸ Feature engineering helpers

In [7]:
THIS_YEAR = datetime.now().year 

# Age & years in dwelling
df_avg["age"] = THIS_YEAR - pd.to_datetime(df_avg["date_of_birth"]).dt.year

df_avg["years_in_dwelling"] = THIS_YEAR - df_avg["move_in_year"].fillna(THIS_YEAR)

# Log‑transforms to soften heavy tails
for col in ["num_claims", "num_cancellations"]:
    df_avg[f"log_{col}"] = np.log1p(df_avg[col])

# Postal FSA (first 3 chars of postal code) 
df_avg["postal_fsa"] = df_avg["postal_code"].str[:3].str.upper()

## 4 ▸ Column lists for the preprocessing pipeline

In [8]:
# High‑cardinality → CV‑safe target encoding
te_cols = ["postal_fsa", "city"]

# Low/medium‑cardinality categoricals → one‑hot
cat_cols = [
    "province", "insurance_type",
    "has_sprinkler_system", "occupants_non_smokers",
    "active_home_insurance", "ever_insured", "multiline_discount",
    "has_monitored_fire_alarm", "has_deadbolt_locks",
    "has_monitored_burglar_alarm",
]

# Numeric features → passthrough
num_cols = [
    "age", "years_in_dwelling", "num_fire_extinguishers",
    "num_mortgages", "occupants", "log_num_claims",
    "log_num_cancellations",
]

In [9]:
# Helper to derive the house/apt flag inside the pipeline
def make_not_apartment(X):
    """Return 1 if *no* unit identifier → standalone dwelling; else 0.
    
    Parameters:
        X (DataFrame): Single-column DataFrame containing the unit_apt column
    Returns:
        DataFrame: Single-column DataFrame with 1 for houses (no unit number) and 0 for apartments
    """
    unit_apt_col = X.iloc[:, 0]  # We know this is the unit_apt column due to ColumnTransformer setup
    return (unit_apt_col.isna() | (unit_apt_col.astype(str).str.strip() == "")).astype(int).to_frame()

## 5 ▸ Assembling the preprocessing pipeline

In [10]:
preprocess = ColumnTransformer(
    transformers=[
        ("te", TargetEncoder(smoothing=0.3), te_cols),  # removed cv parameter and cols parameter
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
        ("not_apt", FunctionTransformer(make_not_apartment, validate=False), ["unit_apt"]),
    ],
    remainder="drop",
)

In [ ]:
df[df["avg_premium"].isna()]

KeyError: 'avg_premium'

## 6 ▸ Train/validation split & model training

In [11]:
# Prepare data
X = df_avg.drop(columns=["avg_premium", "id"])
y = df_avg["avg_premium"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=df_avg["province"]
)

# Define models to test
models = {
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    # 'XGBoost': XGBRegressor(random_state=42, verbosity=0),
    # 'LightGBM': LGBMRegressor(random_state=42, verbose=-1),
    'Lasso': LassoCV(cv=5, random_state=42),
    'Ridge': RidgeCV(cv=5),
    'SVR': SVR(kernel='rbf')
}

# Dictionary to store results
results = {}

# Train and evaluate each model
for name, model in models.items():
    print(f"\nTraining {name}...")
    pipe = Pipeline([("prep", preprocess), ("reg", model)])
    
    # Train the model
    pipe.fit(X_train, y_train)
    
    # Store the pipeline for later use
    results[name] = {
        'pipeline': pipe,
        'train_score': mean_absolute_error(y_train, pipe.predict(X_train)),
        'test_score': mean_absolute_error(y_test, pipe.predict(X_test))
    }

# Create performance summary
performance_df = pd.DataFrame({
    name: {
        'Train MAE': res['train_score'],
        'Test MAE': res['test_score']
    }
    for name, res in results.items()
}).round(2)

print("\nModel Performance Summary (MAE in CAD):")
print(performance_df.T)


Training Gradient Boosting...


ValueError: The target column y must not contain missing values.

In [10]:
# Store the best model based on test score
best_model_name = min(results.items(), key=lambda x: x[1]['test_score'])[0]
pipe = results[best_model_name]['pipeline']
print(f"\nBest performing model: {best_model_name}")


Best performing model: SVR


## 7 ▸ Evaluation

In [11]:
cv_mae = -cross_val_score(pipe, X_train, y_train,
                         cv=5, scoring="neg_mean_absolute_error").mean()

holdout_mae = mean_absolute_error(y_test, pipe.predict(X_test))

print(f"\nCross‑val MAE:  {cv_mae:,.0f} CAD")
print(f"Hold‑out MAE:   {holdout_mae:,.0f} CAD\n")


Cross‑val MAE:  1,404 CAD
Hold‑out MAE:   1,992 CAD



## 8 ▸ Convenience inference wrapper

In [12]:
def predict_average_premium(profile: dict):
    """Return the average premium estimate for a *single* customer profile."""
    # Create DataFrame with single profile
    X_new = pd.DataFrame([profile])
    
    # Feature engineering
    # 1. Calculate age from date of birth
    X_new["age"] = THIS_YEAR - pd.to_datetime(X_new["date_of_birth"]).dt.year
    
    # 2. Calculate years in dwelling
    X_new["years_in_dwelling"] = THIS_YEAR - X_new["move_in_year"].fillna(THIS_YEAR)
    
    # 3. Log transform claims and cancellations
    X_new["log_num_claims"] = np.log1p(X_new["num_claims"])
    X_new["log_num_cancellations"] = np.log1p(X_new["num_cancellations"])
    
    # 4. Extract postal FSA (first 3 chars)
    X_new["postal_fsa"] = X_new["postal_code"].str[:3].str.upper()
    
    # Make prediction
    avg_prediction = pipe.predict(X_new)[0]
    
    # Calculate Senchi discount
    percent_discount = random.uniform(0.25, 0.40)
    senchi_prediction = avg_prediction * percent_discount
    discount = avg_prediction - senchi_prediction
    
    return {
        "avg_premium": avg_prediction,
        "senchi_premium": senchi_prediction,
        "percent_discount": percent_discount,
        "discount": discount
    }

## 9 ▸ Example usage

In [26]:
sample_profile = {
    "postal_code": "M9C 2T6",
    "city": "Toronto",
    "province": "ON",
    "insurance_type": "H",
    "unit_apt": "345",  # empty → house
    "date_of_birth": "2000-07-22",
    "move_in_year": 2015,
    "num_fire_extinguishers": 2,
    "num_mortgages": 1,
    "occupants": 3,
    "num_claims": 0,
    "num_cancellations": 0,
    "has_sprinkler_system": 0,
    "occupants_non_smokers": 0,
    "active_home_insurance": 0,
    "ever_insured": 1,
    "multiline_discount": 0,
    "has_monitored_fire_alarm": 0,
    "has_deadbolt_locks": 1,
    "has_monitored_burglar_alarm": 0,
}

In [27]:
result = predict_average_premium(sample_profile)
print(f"""
Premium Summary:
Average Market Premium:  ${result['avg_premium']:,.2f}
Senchi Premium:         ${result['senchi_premium']:,.2f}
Your Savings:           ${result['discount']:,.2f}
Discount Percentage:    {(result['percent_discount']) * 100:.1f}%
""")


Premium Summary:
Average Market Premium:  $4,007.33
Senchi Premium:         $1,160.71
Your Savings:           $2,846.62
Discount Percentage:    29.0%



## 10 ▸ Persist model

In [ ]:
MODEL_PATH = Path("avg_home_premium_model.joblib")
joblib.dump(pipe, MODEL_PATH)
print(f"Model saved to {MODEL_PATH.resolve()}")